In [19]:
import pickle
import nltk
nltk.download('punkt')

from gpt_ner_api_codes.prompts_all import *
from gpt_ner_api_codes.demonstration_samples_generation import *
%reload_ext autoreload
%autoreload 2

import tqdm
def nop(it, *a, **k):
    return itm
tqdm.tqdm = nop   

from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Thin film head technology dataset

In [20]:
path_original_sentence = "./data/thin-film-technology-dataset/thin_film_head_technology_total_original_sentence.jsonl"
path_entities = "./data/thin-film-technology-dataset/thin_film_head_technology_total_entities.jsonl"
path_tokens_tags = "./data/thin-film-technology-dataset/thin_film_head_technology_total.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 500, demon_size = 300, stratify = True)

(array(['AnnotatorNotes', 'Attribution', 'Component', 'Consequence',
       'Effect', 'EnergyFlow', 'Function', 'InfoFlow', 'Location',
       'Material', 'Measure', 'PhysicsFlow', 'ScientificConcept', 'Shape',
       'State', 'System', 'Value'], dtype='<U1679'), array([   25,  1908, 12911,   132,   750,  1573,  1542,   203,  2601,
        1683,   152,   147,   711,  1074,    40,  1134,   312],
      dtype=int64))
Duplicated few-shot examples:  306
length of stratified_sentences_develop:  283
length of stratified_sentences_test:  500
length of stratified_sentences_train:  3760


In [9]:
with open("data/immutable_data_formal/sen_thf_input_30.txt", "rb") as data:   #Pickling
    sentences_input = pickle.load(data)  
with open("data/immutable_data_formal/dem_thf_input_30.txt", "rb") as data:   #Pickling
    demonstrations_input_solutions = pickle.load(data) 
print(len(sentences_input), len(demonstrations_dev) )
print(demonstrations_dev[2])

500 273
TiW film: Component
3 nm to 5 nm: Value
thickness: Attribution
base film: Component
bias magnet film: Component




#### Long list of words and corresponding entities in development dataset

In [10]:
words_dev = []
words_entities_dev = []
for i, sentence in enumerate(sentences_dev):
    for token_index, token in enumerate(sentence.split(" ")):
        words_dev.append(token)
        #get the list of entities for each sentence
        for demon_row in demonstrations_dev[i].split("\n"):#### Long list of words and corresponding entities in development dataset
            if token in demon_row.split(": ")[0]:
                words_entities_dev.append(demon_row.split(": ")[1])
                break
        if len(words_dev) == len(words_entities_dev)+1:
                words_entities_dev.append("0")  
print(len(words_dev), words_dev[0:10])
print(len(words_entities_dev), words_entities_dev[0:10])

13979 ['(', '1', ')', 'A', 'magnetic', 'recording', 'system', 'is', 'disclosed', ',']
13979 ['0', '0', '0', '0', 'System', 'System', 'System', '0', '0', '0']


#### Long list of words and corresponding entities in input sentence

In [12]:
words_input_sentence = []
for sentence in sentences_input:
    #print(sentence)
    for token in sentence.split(" "):
        words_input_sentence.append(token)
print(len(words_input_sentence))    

17580


In [13]:
similarity_matrix = model.similarity(words_input_sentence, words_dev)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 219/219 [02:18<00:00,  1.58it/s]


In [14]:
top_index_xb = np.argsort( -similarity_matrix, axis=1)
words_dev_array = np.asarray(words_dev)
words_entities_dev_array = np.asarray(words_entities_dev)
words_dev_array[top_index_xb[:,:50]]
ranked_entities = words_entities_dev_array[top_index_xb[:,:50]]
print(ranked_entities)

def most_frequent(List):
    return max(set(List), key = List.count)
ranked_entities = list(ranked_entities)
most_frequent_entities = []
for ranked_entities_one_token in ranked_entities:
    most_frequent_entities.append( most_frequent( list(ranked_entities_one_token)) )
print(len(words_input_sentence) )
print(len(most_frequent_entities)) 
#print(most_frequent_entities)

[['0' '0' '0' ... '0' 'EnergyFlow' '0']
 ['Location' 'Component' 'Component' ... 'Component' 'EnergyFlow'
  'Component']
 ['Component' 'Component' 'Component' ... 'ScientificConcept' 'Component'
  '0']
 ...
 ['Component' 'ScientificConcept' 'Component' ... 'ScientificConcept'
  'Effect' 'ScientificConcept']
 ['0' '0' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']]
17580
17580


In [15]:
# get the output format like gpt 
i = 0
response = []
for sentence_index, sentence in enumerate(sentences_input):
    response_one_sentence = ""
    for token in sentence.split(" "):
        if most_frequent_entities[i] == "0":
            i = i + 1
            continue
        else:
            response_one_sentence = response_one_sentence + token + ": " + most_frequent_entities[i] + "\n"
        i = i + 1
    response.append( response_one_sentence + "\n")    
print(response[0] ) 

pole: Component
pieces: Component
or: Component
core: Component
thin: Component
film: Component
head: Component
electrically: Component
base: EnergyFlow
substrate: Component
thin: Component
film: Component
head: Component




In [16]:
transformed_output = change_output_format_to_tokens_tags(sentences_input, response)
transformed_solution = change_output_format_to_tokens_tags(sentences_input, demonstrations_input_solutions)
get_evaluation_without_o(transformed_solution = transformed_solution, transformed_output = transformed_output) 

['System', 'AnnotatorNotes', 'Material', 'Value', 'ScientificConcept', 'Consequence', 'Shape', 'Location', 'Measure', 'Effect', 'PhysicsFlow', 'InfoFlow', 'Function', 'EnergyFlow', 'Component', 'Attribution', 'State']


,entity,precision,recall,F1 score,support
0,System,39.43662,11.522634,0.178344,243.0
1,AnnotatorNotes,0.0,0.0,0.0,3.0
2,Material,45.180723,23.734177,0.311203,316.0
3,Value,41.37931,20.0,0.269663,60.0
4,ScientificConcept,18.518519,2.747253,0.047847,182.0
5,Consequence,33.333333,4.545455,0.08,22.0
6,Shape,46.808511,11.518325,0.184874,191.0
7,Location,48.375451,28.270042,0.356858,474.0
8,Measure,0.0,0.0,0.0,38.0
9,Effect,27.142857,7.279693,0.114804,261.0


In [28]:
def eva_baseline( path_sen_train, path_dem_train, path_sen_input, path_dem_input ):
        
    with open(path_sen_train, "rb") as data:   #Pickling
        sentences_dev = pickle.load(data)  
    with open(path_dem_train, "rb") as data:   #Pickling
        demonstrations_dev = pickle.load(data)     
    
    with open(path_sen_input, "rb") as data:   #Pickling
        sentences_input = pickle.load(data)  
    with open(path_dem_input, "rb") as data:   #Pickling
        demonstrations_input_solutions = pickle.load(data) 
    print(len(sentences_input), len(demonstrations_dev) )
    print(demonstrations_dev[2])
    
    ########################################
    # long list of words and entities
    ########################################    
    words_dev = []
    words_entities_dev = []
    for i, sentence in enumerate(sentences_dev):
        for token_index, token in enumerate(sentence.split(" ")):
            words_dev.append(token)
            #get the list of entities for each sentence
            for demon_row in demonstrations_dev[i].split("\n"):#### Long list of words and corresponding entities in development dataset
                if token in demon_row.split(": ")[0]:
                    words_entities_dev.append(demon_row.split(": ")[1])
                    break
            if len(words_dev) == len(words_entities_dev)+1:
                    words_entities_dev.append("0")  
    print(len(words_dev), words_dev[0:10])
    print(len(words_entities_dev), words_entities_dev[0:10])
    
    ########################################
    # Long list of words and corresponding entities in input sentence
    ########################################  
    # words_input_sentence = []
    for sentence in sentences_input:
        #print(sentence)
        for token in sentence.split(" "):
            words_input_sentence.append(token)
    print(len(words_input_sentence)) 
    
    
    ############## use CSE to measure similarities
    similarity_matrix = model.similarity(words_input_sentence, words_dev)
    
    top_index_xb = np.argsort( -similarity_matrix, axis=1)
    words_dev_array = np.asarray(words_dev)
    words_entities_dev_array = np.asarray(words_entities_dev)
    words_dev_array[top_index_xb[:,:50]]
    ranked_entities = words_entities_dev_array[top_index_xb[:,:50]]
    #print(ranked_entities)

    def most_frequent(List):
        return max(set(List), key = List.count)
    ranked_entities = list(ranked_entities)
    most_frequent_entities = []
    for ranked_entities_one_token in ranked_entities:
        most_frequent_entities.append( most_frequent( list(ranked_entities_one_token)) )
    print(len(words_input_sentence) )
    print(len(most_frequent_entities)) 
    #print(most_frequent_entities)
    

    ########################################
    # get the output format like gpt 
    ########################################      
    i = 0
    response = []
    for sentence_index, sentence in enumerate(sentences_input):
        response_one_sentence = ""
        for token in sentence.split(" "):
            if most_frequent_entities[i] == "0":
                i = i + 1
                continue
            else:
                response_one_sentence = response_one_sentence + token + ": " + most_frequent_entities[i] + "\n"
            i = i + 1
        response.append( response_one_sentence + "\n")    
    #print(response[0] ) 
    
    ########## evaluation
    transformed_output = change_output_format_to_tokens_tags(sentences_input, response)
    transformed_solution = change_output_format_to_tokens_tags(sentences_input, demonstrations_input_solutions)
    get_evaluation_without_o(transformed_solution = transformed_solution, transformed_output = transformed_output) 
    
    

In [29]:
path_sen_train = "data/immutable_data_formal/sen_thf_train.txt"
path_dem_train = "data/immutable_data_formal/dem_thf_train.txt"
path_sen_input = "data/immutable_data_formal/sen_thf_input_30.txt"
path_dem_input = "data/immutable_data_formal/dem_thf_input_30.txt"
eva_baseline( path_sen_train, path_dem_train, path_sen_input, path_dem_input )

500 3760
magnetic recording system: Component
thin-film magnetic head: Component
magnetic recording medium: Material
nonmagnetic support: Component
nonmagnetic layer: Component
inorganic nonmagnetic particles: Material
binder resin: PhysicsFlow
nonmagnetic support: Component
magnetic layer: Component
mainly ferromagnetic metal particles: Material
binder: PhysicsFlow
resin: Material
nonmagnetic layer: Component
binder resins: PhysicsFlow
magnetic and nonmagnetic layers: Component
vinyl chloride resin: PhysicsFlow
ferromagnetic metal particles: Material
organic compound: Material
magnetic layer: Component
thickness: Attribution
0.05 to 1.0: Value


130029 ["''", ')', 'is', 'then', 'deposited', 'over', 'the', 'MSL', 'layer', 'and']
130029 ['0', '0', '0', '0', '0', '0', '0', 'Component', 'Component', '0']
49641


100%|█████████████████████████████████████████████████████████████████████| 2032/2032 [17:52<00:00,  1.89it/s]


MemoryError: Unable to allocate 48.1 GiB for an array with shape (49641, 130029) and data type int64

# Assembly

In [17]:
path_original_sentence = "./data/assembly_dataset/alternators-engines-gearboxes/train_original_sentence.jsonl"
path_entities = "./data/assembly_dataset/alternators-engines-gearboxes/train_entities.jsonl"
path_tokens_tags = "./data/assembly_dataset/alternators-engines-gearboxes/train.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 500, demon_size = 300, stratify = True)

(array(['DIM', 'ID', 'OPER', 'PART', 'PROP', 'QTY', 'RPOS', 'TOOL', 'WGT'],
      dtype='<U650'), array([  55,  101,  735, 2528,   84,   92,  699,  108,    2], dtype=int64))
Duplicated few-shot examples:  300
length of stratified_sentences_develop:  239
length of stratified_sentences_test:  399
length of stratified_sentences_train:  776


In [26]:
with open("data/immutable_data_formal/sen_assembly_input_30.txt", "rb") as data:   #Pickling
    sentences_input = pickle.load(data)  
with open("data/immutable_data_formal/dem_assembly_input_30.txt", "rb") as data:   #Pickling
    demonstrations_input_solutions = pickle.load(data)  

In [27]:
words_dev = []
words_entities_dev = []
for i, sentence in enumerate(sentences_dev):
    for token_index, token in enumerate(sentence.split(" ")):
        words_dev.append(token)
        #get the list of entities for each sentence
        for demon_row in demonstrations_dev[i].split("\n"):#### Long list of words and corresponding entities in development dataset
            if token in demon_row.split(": ")[0]:
                words_entities_dev.append(demon_row.split(": ")[1])
                break
        if len(words_dev) == len(words_entities_dev)+1:
                words_entities_dev.append("0")  
print(len(words_dev), words_dev[0:10])
print(len(words_entities_dev), words_entities_dev[0:10])

7319 ['#', 'Install', 'guide', '0', '\n', '&', 'into', 'exhaust', 'rear', '769938']
7319 ['ID', 'OPER', '0', '0', '0', '0', 'RPOS', 'PART', 'RPOS', 'ID']


In [28]:
words_input_sentence = []
for sentence in sentences_input:
    #print(sentence)
    for token in sentence.split(" "):
        words_input_sentence.append(token)
print(len(words_input_sentence))    

11190


In [29]:
similarity_matrix = model.similarity(words_input_sentence, words_dev)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 115/115 [01:11<00:00,  1.61it/s]


In [30]:
top_index_xb = np.argsort( -similarity_matrix, axis=1)
words_dev_array = np.asarray(words_dev)
words_entities_dev_array = np.asarray(words_entities_dev)
words_dev_array[top_index_xb[:,:50]]
ranked_entities = words_entities_dev_array[top_index_xb[:,:50]]
print(ranked_entities)

def most_frequent(List):
    return max(set(List), key = List.count)
ranked_entities = list(ranked_entities)
most_frequent_entities = []
for ranked_entities_one_token in ranked_entities:
    most_frequent_entities.append( most_frequent( list(ranked_entities_one_token)) )
print(len(words_input_sentence) )
print(len(most_frequent_entities)) 
#print(most_frequent_entities)

[['ID' '0' 'ID' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']
 ...
 ['PROP' 'PROP' '0' ... 'ID' '0' 'ID']
 ['0' 'ID' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']]
11190
11190


In [31]:
# get the output format like gpt 
i = 0
response = []
for sentence_index, sentence in enumerate(sentences_input):
    response_one_sentence = ""
    for token in sentence.split(" "):
        if most_frequent_entities[i] == "0":
            i = i + 1
            continue
        else:
            response_one_sentence = response_one_sentence + token + ": " + most_frequent_entities[i] + "\n"
        i = i + 1
    response.append( response_one_sentence + "\n")    
print(response[0] ) 

kit: PART
Install: OPER




In [32]:
transformed_output = change_output_format_to_tokens_tags(sentences_input, response)
transformed_solution = change_output_format_to_tokens_tags(sentences_input, demonstrations_input_solutions)
get_evaluation_without_o(transformed_solution = transformed_solution, transformed_output = transformed_output)

['TOOL', 'OPER', 'RPOS', 'ID', 'PART', 'PROP', 'WGT', 'DIM', 'QTY']


,entity,precision,recall,F1 score,support
0,TOOL,0.0,0.0,0.0,72.0
1,OPER,62.795276,60.877863,0.618217,524.0
2,RPOS,67.706013,62.55144,0.650267,486.0
3,ID,100.0,1.538462,0.030303,65.0
4,PART,74.459273,78.164251,0.762668,2070.0
5,PROP,0.0,0.0,0.0,55.0
6,WGT,0.0,0.0,0.0,2.0
7,DIM,0.0,0.0,0.0,35.0
8,QTY,0.0,0.0,0.0,61.0
0,accuracy,-,-,66.52819,3370.0


In [ ]:
path_sen_train = "data/immutable_data_formal/sen_assembly_train.txt"
path_dem_train = "data/immutable_data_formal/dem_assembly_train.txt"
path_sen_input = "data/immutable_data_formal/sen_assembly_input_30.txt"
path_dem_input = "data/immutable_data_formal/dem_assembly_input_30.txt"
eva_baseline( path_sen_train, path_dem_train, path_sen_input, path_dem_input )

# FabNER

In [11]:
path_original_sentence = "./data/fabNER/fabner_simple_total_original_sentence.jsonl"
path_entities = "./data/fabNER/fabner_simple_total_entities.jsonl"
path_tokens_tags = "./data/fabNER/fabner_simple_total.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 500, demon_size = 300, stratify = True)

(array(['APPL', 'BIOP', 'CHAR', 'CONPRI', 'ENAT', 'FEAT', 'MACEQ', 'MANP',
       'MANS', 'MATE', 'PARA', 'PRO'], dtype='<U1044'), array([ 2210,   236,  2226, 21113,  1238,  2610,  2551, 10622,    92,
       12881,  3477,  5117], dtype=int64))
Duplicated few-shot examples:  312
length of stratified_sentences_develop:  309
length of stratified_sentences_test:  500
length of stratified_sentences_train:  13658


In [12]:
with open("data/immutable_data_formal/sen_fabner_input_30.txt", "rb") as data:   #Pickling
    sentences_input = pickle.load(data)  
with open("data/immutable_data_formal/dem_fabner_input_30.txt", "rb") as data:   #Pickling
    demonstrations_input_solutions = pickle.load(data)  

In [13]:
words_dev = []
words_entities_dev = []
for i, sentence in enumerate(sentences_dev):
    for token_index, token in enumerate(sentence.split(" ")):
        words_dev.append(token)
        #get the list of entities for each sentence
        for demon_row in demonstrations_dev[i].split("\n"):#### Long list of words and corresponding entities in development dataset
            if token in demon_row.split(": ")[0]:
                words_entities_dev.append(demon_row.split(": ")[1])
                break
        if len(words_dev) == len(words_entities_dev)+1:
                words_entities_dev.append("0")  
print(len(words_dev), words_dev[0:10])
print(len(words_entities_dev), words_entities_dev[0:10])

9814 ['%', ')', 'was', 'mixed', 'in', 'AlSi10Mg', 'powder', 'by', 'conducting', 'low-energy']
9814 ['0', '0', '0', '0', 'MANP', 'MATE', '0', '0', '0', '0']


In [14]:
words_input_sentence = []
for sentence in sentences_input:
    #print(sentence)
    for token in sentence.split(" "):
        words_input_sentence.append(token)
print(len(words_input_sentence))  

14481


In [15]:
similarity_matrix = model.similarity(words_input_sentence, words_dev)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 154/154 [01:04<00:00,  2.37it/s]


In [16]:
top_index_xb = np.argsort( -similarity_matrix, axis=1)
words_dev_array = np.asarray(words_dev)
words_entities_dev_array = np.asarray(words_entities_dev)
words_dev_array[top_index_xb[:,:50]]
ranked_entities = words_entities_dev_array[top_index_xb[:,:50]]
print(ranked_entities)

def most_frequent(List):
    return max(set(List), key = List.count)
ranked_entities = list(ranked_entities)
most_frequent_entities = []
for ranked_entities_one_token in ranked_entities:
    most_frequent_entities.append( most_frequent( list(ranked_entities_one_token)) )
print(len(words_input_sentence) )
print(len(most_frequent_entities)) 
#print(most_frequent_entities)

[['0' '0' '0' ... '0' '0' '0']
 ['0' 'FEAT' 'FEAT' ... '0' 'PARA' '0']
 ['0' '0' '0' ... 'CONPRI' '0' '0']
 ...
 ['0' '0' '0' ... 'CONPRI' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']]
14481
14481


In [17]:
# get the output format like gpt 
i = 0
response = []
for sentence_index, sentence in enumerate(sentences_input):
    response_one_sentence = ""
    for token in sentence.split(" "):
        if most_frequent_entities[i] == "0":
            i = i + 1
            continue
        else:
            response_one_sentence = response_one_sentence + token + ": " + most_frequent_entities[i] + "\n"
        i = i + 1
    response.append( response_one_sentence + "\n")    
print(response[0] ) 

a: CONPRI
stages: CONPRI
technology: CONPRI




In [18]:
transformed_output = change_output_format_to_tokens_tags(sentences_input, response)
transformed_solution = change_output_format_to_tokens_tags(sentences_input, demonstrations_input_solutions)
get_evaluation_without_o(transformed_solution = transformed_solution, transformed_output = transformed_output)

['MANP', 'CHAR', 'ENAT', 'PRO', 'BIOP', 'CONPRI', 'MATE', 'MACEQ', 'PARA', 'APPL', 'MANS', 'FEAT']


,entity,precision,recall,F1 score,support
0,MANP,47.928994,34.177215,0.399015,711.0
1,CHAR,75.555556,17.435897,0.283333,195.0
2,ENAT,12.5,1.923077,0.033333,52.0
3,PRO,42.990654,21.800948,0.289308,211.0
4,BIOP,11.764706,22.222222,0.153846,9.0
5,CONPRI,39.332366,28.952991,0.333538,936.0
6,MATE,56.774194,48.708487,0.52433,542.0
7,MACEQ,62.068966,14.173228,0.230769,127.0
8,PARA,26.229508,8.205128,0.125,195.0
9,APPL,15.686275,8.888889,0.113475,90.0


In [ ]:
path_sen_train = "data/immutable_data_formal/sen_fabner_train.txt"
path_dem_train = "data/immutable_data_formal/dem_fabner_train.txt"
path_sen_input = "data/immutable_data_formal/sen_fabner_input_30.txt"
path_dem_input = "data/immutable_data_formal/dem_fabner_input_30.txt"
eva_baseline( path_sen_train, path_dem_train, path_sen_input, path_dem_input )